# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
Since the goal is to identify students who might need early intervention, our predictor variable is discrete—the “pass" variable. By training data on students that passed or not, the model will be able to predict which students will pass or not pass, allowing administrators the ability to identify candidates for early intervention before they actually fail. Various classification algorithms will be trained with cross validation to compare the predictive accuracy of the classifiers. With the decision boundary identified, we can predict the class label for new data. Furthermore, regression would not be appropriate for this problem because the response variable is continuous, and will is not suitable for classifying students by the discrete class variable “pass.” 



## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students

n_students = len(student_data.index)

# TODO: Calculate number of features

n_features = len(student_data.columns)

# TODO: Calculate passing students
n_passed = len(student_data[(student_data["passed"]=="yes")])
# student_data['passed'].value_counts()

# TODO: Calculate failing students
n_failed = len(student_data[(student_data["passed"]=="no")])

# TODO: Calculate graduation rate
grad_rate = float (n_passed) / (n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 0.67%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:\n", student_data.head(n=5)


Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

   ...   internet romantic  famrel  freetime  goout Dalc Walc health absences  \
0  ...         no       no       4         3      4    1    1      3        6   
1  ...        

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.24, random_state = 42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application






*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
In an Efficient Multi-Relational Naïve Bayesian Classifier based on Semantic Relationship Graph, Liu argues that multi-relational Naïve Bayesian classifiers work well at predicting a borrower’s ability to repay loans. Peer-to-Peer (P2P) lending platforms predict borrower’s default risk by merging social network data with traditional credit data; however, when social network data is merged from several sources into one data table, the single data table suffers from high redundancy. Bayesian classification methods can handle high redundancy, which is why Naïve Bayes classifiers are used in the P2P lending industry. 
The Naïve Bayes classifier has several strengths: it is easy to implement, well known and understood, and empirically successful. If independence of attributes holds, NB classifier will converge quicker than discriminative models thus requiring less training data. 
Naïve Bayes biggest disadvantage is its simplicity. At times, model complexity is advantageous when generalizing the relationship of from complex relationships and interactions. Furthermore, the assumption of independence among attributes may not be realistic and models that do not force this assumption on the class structure can better perform when there is high dependence among attributes. According to Haste, even if the individual class density estimates are biased, the posterior probabilities near the decision can withstand considerable bias—the posterior probabilities smooth can be smooth even if the population class densities are not (pg. 210-211).
Naïve Bayes was a good candidate because it the data set was small, thus a more complex classifier may over fit compared to Naïve Bayes. According to a Udacity lecturer, Naïve Bayes is a good first model that can provide a reasonable baseline to compare other models against. 

According to Bergstra, AdaBoost methods have been proven adept within the music streaming industry to classify digital music.  Given a set of audio features that have been extracted from segments of songs, the AdaBoost classifier can be trained to predict both the musical genre and artist. Interestingly, the authors provide evidence that AdaBoost classification works better if the classifier is trained on only segments of audio, not entire songs or individual sort-timescale features. By accurately classifying a listener’s taste in music, digital streaming services can provide more accurate suggestions (Apple’s Genius) to customers seeking new artists. 
The AdaBoost classifier has several strengths: it is computationally efficient, the parameters are easy to set, and crucially, a wide array of weak learners may be used including, the AdaBoost, Bernoulli NB, DecisionTree, Multinomial N.B., NuSVC, Perceptron, Random Forest, Ridge Classifier and Support Vector Machines. 
The AdaBoost classifier has several weaknesses: it is susceptible to uniform noise, the base learners should be weak learners in order to avoid overfitting and a sufficient amount of data is necessary utilize weak learner’s ability to make use of marginal improvements that in aggregate provide complexity. 
AdaBoost classifier was a good candidate because Boosting can create complex classifiers through weak learners—thus allowing AdaBoost to iteratively focus on the data points that are more difficult to classify.  

According to Kaiyi, the Support Vector Machines (SVM) classifier has been used in New Zealand to automate the hazard analysis and critical control points (HACCP) to improve food safety. To implement the first and second principles of the HACCP, the SVM classifies hazards into critical control points (CCPs) and non-CCPs. SVM have shown considerable advantages in this complex structured task of food safety. 
The SVM classifier has multiple strengths: it is effective in high dimensional spaces and in cases where the number of dimension is greater than the number of samples, it is memory efficient because it concentrates on a subset of training points near the decision boundary (support vectors) rather than all the data points. 
The SVM classifier has several disadvantages: given a large data set, the SVM performs requires a longer training time, noisy data (overlapping target classes) can cause SVM to perform worse than Naïve Bayes classifiers and given a lot of features, it will perform slowly and tend to over fit the data. 
The SVM classifier was a good candidate because SVM do well with complicated domains with marginal separations at the decision boundary—since we are trying to maximize the margin with SVMs.  Additionally, since the data was not very large or feature rich, SVM classifier was a good candidate. 
 

References:

H. Liu, X. Yin and J. Han, "An Efficient Multirelational Naive Bayesian Classifier Based on Semantic Relationship Graph", ACM MRDM, 2005.
Wang Kaiyi, Zhao Chunjiang, Zhang Fang Tian, Wang Xiao Juan, Chen Hong, and Yao Yuan Fang, New Zealand Journal Of Agricultural Research Vol. 50 , Iss. 5,2007
Bergstra, J., Casagrande, N., Erhan, D. et al. Mach Learn (2006) 65: 473. doi:10.1007/s10994-006-9019-7
Ray, Sunil. Understanding Support Vector machine algorithm form examples (along with code). Analytics Vidya. N.p. 6 Oct. 2015. Web. 25 Sept. 2016. 
Hastie, Trevor, Trevor Hastie, Robert Tibshirani, and J H. Friedman. The Elements of Statistical Learning: Data Mining, Inference, and Prediction. New York: Springer, 2001. Print.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)
    
    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn

# from sklearn import model_A
from sklearn.naive_bayes import GaussianNB

# from sklearn import model_B
from sklearn.ensemble import AdaBoostClassifier

# from skearln import model_C
from sklearn.svm import SVC

# TODO: Initialize the three models
classifiers = [
    GaussianNB(),
    AdaBoostClassifier(random_state=43),
    SVC(random_state=43),]

# TODO: Set up the training set sizes

X_train_100 = X_train.iloc[:100]
y_train_100 = y_train.iloc[:100]

X_train_200 = X_train.iloc[:200]
y_train_200 = y_train.iloc[:200]

X_train_300 = X_train.iloc[:300]
y_train_300 = y_train.iloc[:300]


datasets = [X_train_100, y_train_100, X_train_200, y_train_200,
           X_train_300, y_train_300]

#iterate over data sets


# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

for clf in classifiers: 
    train_predict(clf,X_train_100, y_train_100, X_test, y_test)
    train_predict(clf,X_train_200, y_train_200, X_test, y_test)
    train_predict(clf,X_train_300, y_train_300, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0027 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8467.
Made predictions in 0.0009 seconds.
F1 score for test set: 0.8029.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0011 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8406.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7244.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0018 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 0.8038.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7634.
Training a AdaBoostClassifier using a training set size of 100. . .
Trained model in 0.1944 seconds
Made predictions in 0.0092 seconds.
F1 score for training set: 0.9481.
Made predictions in 0.0065 seconds.
F1 score for test set: 0.7669.
Training a AdaBoostClassifier using a training set size of 200. . .
Trai

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            .0028             |   0.0005                     |      0.8467            |     0.8029            |
| 200               |        0.0012          |          0.0003              |         0.8406         |    0.7244             |
| 300               |            0.0017          |      0.0005                  |      0.8038            |    0.7634      |

** Classifer 2 - AdaBoost **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.1452             |       0.0067             |      0.9481            |        0.7669         |
| 200               |     0.1499  |           0.0062    |  0.8927     |         0.8281        |
| 300               |     0.1395                    |    0.0061                    |   0.8637               |     0.7829     |

** Classifer 3 - SVC **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0015                   |    0.0008                    |        0.8777          |       0.7746          |
| 200               |  0.0022            |       0.0011        | 0.8679                 |      0.7815            |
| 300               |   0.0067          |      0.0023                  |         0.8761         |     0.7838            |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

After testing several models to accurately predict whether a student will pass or fail, the Support Vector Machine was chosen based primarily on the available data, performance, and resources required. The size of the data available was quite small—only 395 data points.  SVMs are known to perform well on small data sets. The SVM classifier steadily improved the F1 score, a weighted average of the precision and recall tested, when given more training samples. This is encouraging because we can likely improve the F1 score further if we collect and train the data on a larger data set. The resource required (time) to train the model was much lower for the SVM than the AdaBoost classifier—giving SVM an advantage over the AdaBoost classifier even though the F1 scores were about the same. Although the Gaussian NB classifier required even less time than the SCM classifier, it fell short in terms of F1 score. 



### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

   To explain the SVM, I will ask the Board to consider the data we already have on students.  The “pass” variable is what we would like to predict (output variable = response), while all the other variables are used to make those predictions (input variables = features.) We devise a way to place students on the basketball court so that students with the most similar features are grouped closest to one another. Then, based on the response variable, we give students who passed a green shirt and those that failed a red shirt. Our goal is take a rope and pass it between students on the basketball court to separate them by shirt color—maximizing the distance between the rope and students closest to the rope (the margin.)  

   If the original data of students is representative of the population, the rope will allow us to place new students on the basketball court based on the same features. Once we look at where the students are standing, we can predict who will pass or fail based on what side of the rope they are on. The SVM works much the same way, implementing what is known as the kernel-trick to allow the rope to curve between students. The SVM will implement cross-validation, which is a fancy way to avoid overfitting by holding back some students (testing set) that will be used to evaluate how well the model classifies students (training set). We will randomly subset the students up into 10 smaller groups and allow each group to be the testing set for the 9 other groups in order to aggregate the 9 different ropes that are made to avoid overfitting. 



### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [9]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit

#Create Cross coss-validation sets from the training set
cv_sets = ShuffleSplit(X_train.shape[0], n_iter = 10, test_size = .20, 
                      random_state = 0)

# TODO: Create the parameters list you wish to tune
parameters = {'degree': [1,2,3], 'kernel': ['poly', 'sigmoid', 'rbf'], 
            'C' : [.75,1.0,1.10], 
            'gamma': [.030,.032, .033,0.036], 
            'cache_size': [90, 100,200]}


# TODO: Initialize the classifier
clf = SVC(random_state=43)


# TODO: Make an f1 scoring function using 'make_scorer' 
def score_func(y_true, y_pred):
    # calculate the f1 score given the predicted and expected labels
    return f1_score(y_true,y_pred, pos_label='yes')

f1_scorer = make_scorer(score_func)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, param_grid = parameters, scoring = f1_scorer, cv = cv_sets)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf
# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

SVC(C=1.0, cache_size=90, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=1, gamma=0.032, kernel='poly',
  max_iter=-1, probability=False, random_state=43, shrinking=True,
  tol=0.001, verbose=False)
Made predictions in 0.0050 seconds.
Tuned model has a training F1 score of 0.8323.
Made predictions in 0.0014 seconds.
Tuned model has a testing F1 score of 0.7945.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
The tuned model’s training F1 score is 0.8323 compared to the untuned model’s 0.8761.
The tuned model’s test F1 score is 0.7945 compared to the untuned model’s 0.7838—a modest improvement of 1.07 % over the untuned model. This modest improvement demonstrates the advantages of using Grid Search to fine tune parameters rather than to accept the default parameters of the model. Finally, the convergence of the tuned model’s training and testing F1 score demonstrates that by tuning the parameters, we can reduce some of the overfitting. 


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.